## Using MoSH to fit body models to labeled mocaps

We base this off of the two SOMA tutorials: [Run SOMA on SOMA dataset](https://github.com/nghorbani/soma/blob/main/src/tutorials/run_soma_on_soma_dataset.ipynb) and [Solve Already Labeled MoCaps with Mosh++](https://github.com/nghorbani/soma/blob/main/src/tutorials/solve_labeled_mocap.ipynb).

Reading the [definitions](https://github.com/nghorbani/soma/tree/main/src/tutorials#definitions) of terminology is recommended.

MoSH++ takes in labeled motion capture data and fits an SMPL body model to it. As a note, the original MoSH paper converted the motion capture data to a different body model; we use MoSH/MoSH++ interchangeably, with both referring to the MoSH++.

A *labeled* motion capture is simply one that has markers assigned to the point cloud points for each frame. The exact names of the markers is not relevant, just their stability (Is the marker mapped to the same point consistently for each frame)  (TODO: verify this).

From the [MoSH paper](https://files.is.tue.mpg.de/black/papers/MoSh.pdf):

> Marker placement on the human body varies across subjects and sessions, consequently we do not assume that the exact marker placement is known. Instead, a key contribution of MoSh is that it solves for the observed marker locations relative to the 3D body model.

The MoSH algorithm works in two steps:

1. Solving for Beta Parameters. A single frame from the sequence is picked, and the SMPL body model is fit to the frame. SMPL models, in their simplest form, have 82 parameters per frame: 23 joints, 3 parameters to represent each joint in 3d space with axis-angle rotation, 3 more parameters to represent the root of the body (global translation), and 10 Beta parameters to characterize the variation of body height, body proportion, and weight. Once these Beta Parameters are solved for, they are fixed, and are used to fit the rest of the frames in the sequence. This process can take 20-30minutes per subject.
2. With Betas fixed, MoSH++ fits the SMPL body model frame by frame for the sequence. This process takes ~2 seconds per frame.

TODO: cite the paper that has the approximate times for each step.

TODO: bring in info about how MoSH uses temporal smoothing for its frame-by-frame fitting.


### Verify Environment

In [1]:
# If this doesn't work, something is wrong with the conda environment. Check conda-env.md
# Make sure you are using the right kernel
import soma
import psbody
import moshpp

In [2]:
from glob import glob
import os
import os.path as osp
import time

from loguru import logger

from soma.amass.mosh_manual import mosh_manual

## Running MoSH

First, we define the directories we will be using to find our mocaps, and where to store them as well.
There are two ways to run MoSH: one, on already labeled mocap data, and two, from data that was labeled from SOMA. We will cover both.
In this first way, we assume we have a folder of already labeled mocaps, and we simply pass folder(s) into `target_ds_names`. Ensure that each such folder has a `settings.json` with the gender information of the mocap data.

In [3]:
soma_work_base_dir = osp.join(os.getcwd(), 'soma-root')
support_base_dir = osp.join(soma_work_base_dir, 'support_files')

mocap_base_dir = osp.join(support_base_dir, 'evaluation_mocaps/original')

work_base_dir = osp.join(soma_work_base_dir, 'running_just_mosh')
temp_base_dir = osp.join(support_base_dir, 'render_out_temp')

# list each of the dataset folders you want to process with MoSH
# Verify that the location of these folders is at TODO: fill this in
# For raw, unlabeled mocaps, make sure to run the labeling script first at `label.ipynb`
# Run on 'immersion_test' to see sample results on a few mocaps
target_ds_names = ['fsp02',]

### View list of Mocaps to MoSH


In [4]:
def select(fname):
    '''
    Running MoSH takes time, so a filter can be applied if only a subset of mocaps is needed.
    '''
    # add filters here if needed
    return True

In [5]:
# List out all filenames we want to MoSH.
for ds_name in target_ds_names:
    mocap_fnames = glob(osp.join(mocap_base_dir, ds_name,  '*/*.c3d'))

    mocap_fnames = list(filter(select, mocap_fnames)) # by default, `select()` does not filter anything
    logger.info(f'#mocaps found for {ds_name}: {len(mocap_fnames)}')
    logger.info(f'mocaps:\t{mocap_fnames}...')

2023-04-25 06:29:55.213 | INFO     | __main__:<module>:6 - #mocaps found for fsp02: 1
2023-04-25 06:29:55.214 | INFO     | __main__:<module>:7 - mocaps:	['/home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/fsp02/20220701_01_001/se001_EnGarde_001_engarde_LH_action31.c3d']...



Pay special attention to the `mosh_stagei_perseq` variable. If true, EACH mocap will have its body solved independently (30-40 mins per mocap). If false, the body will be solved once, and used for all the data.
Set `true` if the data is from different subjects -- or different markerlayouts -- or different sessions. The only time we should do false is for mocap data recorded in the same session, where no markers have moved (or fallen off/re-added in different places). 

In [7]:
for ds_name in target_ds_names:
    mocap_fnames = glob(osp.join(mocap_base_dir, ds_name,  '*/*.c3d'))

    mocap_fnames = list(filter(select, mocap_fnames))

    logger.info(f'#mocaps found for {ds_name}: {len(mocap_fnames)}')

    mosh_manual(
        mocap_fnames,
        mosh_cfg={
            'moshpp.verbosity': 1, # set to 2 to visualize the process in meshviewer
            'dirs.work_base_dir': osp.join(work_base_dir, 'mosh_results'),
            'dirs.support_base_dir': support_base_dir,
            'mosh_stagei_perseq': True,
            'moshpp.stagei_frame_picker.least_avail_markers': 0.8,
        },
        parallel_cfg={
            'pool_size': 5,
            # 'max_num_jobs': 1, # comment on real runs
            'randomly_run_jobs': True,
        },
        run_tasks=[
            'mosh',
        ],
        fast_dev_run=False,
    )

20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- 457002152:<module>:6 -- #mocaps found for fsp02: 1
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- mosh_manual:mosh_manual:107 -- Submitting MoSh++ jobs.
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- parallel_tools:run_parallel_jobs:54 -- #Job(s) submitted: 1
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- parallel_tools:run_parallel_jobs:67 -- Will run the jobs in random order.
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- mosh_head:__init__:95 -- mocap_fname: /home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/fsp02/20220701_01_001/se001_EnGarde_001_engarde_LH_action31.c3d
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- mosh_head:__init__:97 -- stagei_fname: /home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/fsp02/20220701_01_001/male_stagei.pkl
20220701_01_001 -- se001_EnGarde_001_engard

/home/ritaank/miniconda3/envs/soma3/lib/python3.7/site-packages/omegaconf/resolvers/oc/__init__.py:91: UserWarning: 'dirs.markerlyout_basename' is deprecated. Change your code and config to use 'dirs.marker_layout.basename'
  warnings.warn(category=UserWarning, message=msg)
/home/ritaank/miniconda3/envs/soma3/lib/python3.7/site-packages/omegaconf/resolvers/oc/__init__.py:91: UserWarning: 'dirs.marker_layout_fname' is deprecated. Change your code and config to use 'dirs.marker_layout.fname'
  warnings.warn(category=UserWarning, message=msg)


20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- edit_tools:marker_layout_load:130 -- Loading marker layout: /home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/fsp02/20220701_01_001/male_stagei.json
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- edit_tools:marker_layout_load:134 -- Available marker types: {'body': 39}. Total: 39 markers.
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- edit_tools:marker_layout_load:130 -- Loading marker layout: /home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/fsp02/20220701_01_001/male_stagei.json
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- edit_tools:marker_layout_load:134 -- Available marker types: {'body': 39}. Total: 39 markers.
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- mosh_head:dump_stagei_marker_layout:338 -- created /home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/fsp02/20220

/home/ritaank/miniconda3/envs/soma3/lib/python3.7/site-packages/omegaconf/resolvers/oc/__init__.py:91: UserWarning: 'dirs.markerlyout_basename' is deprecated. Change your code and config to use 'dirs.marker_layout.basename'
  warnings.warn(category=UserWarning, message=msg)
/home/ritaank/miniconda3/envs/soma3/lib/python3.7/site-packages/omegaconf/resolvers/oc/__init__.py:91: UserWarning: 'dirs.marker_layout_fname' is deprecated. Change your code and config to use 'dirs.marker_layout.fname'
  warnings.warn(category=UserWarning, message=msg)


20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- mosh_head:mosh_stageii:298 -- finished mosh stageii in 0:27:17.288175
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- mosh_head:run_moshpp_once:606 -- Final mosh stageii loss: data = 2.87e+07 | poseB = 1.06e+08 | velo = 2.26e-01


## Rendering outputs

After our files have been MoSHed, we can render the output SMPL mesh as an mp4, a series of pngs, and/or a blender file.
We repeat a similar process to above, changing the `run_task` to render, as well as defining different configuration variables. We also need to define a `temp_base_dir` to store the intermediate rendering files.

In [9]:
for ds_name in target_ds_names:
    mocap_fnames = glob(osp.join(mocap_base_dir, ds_name,  '*/*.c3d'))

    mocap_fnames = list(filter(select, mocap_fnames))

    logger.info(f'#mocaps found for {ds_name}: {len(mocap_fnames)}')

    mosh_manual(
        mocap_fnames,
        mosh_cfg={
            # 'moshpp.verbosity': 1, # set to 2 to visualize the process in meshviewer
            'dirs.work_base_dir': osp.join(work_base_dir, 'mosh_results'),
            'dirs.support_base_dir': support_base_dir,
            # 'mosh_stagei_perseq': True,
        },
        render_cfg={
            # 'render.render_only_one_image': True, # uncomment for initial testing of the pipeline
            'render.video_fps': 30,  # video_fps * ds_rate should equal the sample fps always
            'mesh.ds_rate': 8, # see above
            'render.resolution.change_from_blend': True,
            'render.resolution.default': [1600, 1600],  # [x,y]
            'render.render_engine': 'eevee',  # eevee / cycles,
        
            'dirs.work_base_dir': osp.join(work_base_dir, 'mp4_renders'),
            # 'render.show_markers': True,
            'render.save_final_blend_file': True,
            'dirs.support_base_dir': support_base_dir,
            'dirs.temp_base_dir': temp_base_dir
        },
        parallel_cfg={
            'pool_size': 5,
            'max_num_jobs': 2,
            'randomly_run_jobs': True,
        },
        run_tasks=[
            'render',
        ],
        fast_dev_run=False,
    )

20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- 2915174843:<module>:6 -- #mocaps found for fsp02: 1
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- mosh_manual:mosh_manual:114 -- Submitting render jobs.
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- parallel_tools:run_parallel_jobs:54 -- #Job(s) submitted: 1
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- parallel_tools:run_parallel_jobs:58 -- max_num_jobs is set to 2. choosing the first #Job(s): 1
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- parallel_tools:run_parallel_jobs:67 -- Will run the jobs in random order.
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- parameters_to_mesh:convert_to_mesh_once:61 -- Preparing mesh files for: ['/home/ritaank/Documents/dev/SOMA-interface/soma-root/running_just_mosh/mosh_results/fsp02/20220701_01_001/se001_EnGarde_001_engarde_LH_action31_stageii.pkl']
20220701_01_001 -- se001_EnGarde_001_engarde_LH_action31 -- parameters_t

: 

: 